# Sentimental Analysis
This part is a combination of twitter api and google NLP api.
The main focus is to extract tweets from twitter handle and use google NLP sevice to analyse them.

## twitter part

In [257]:
import requests
import pandas as pd
import json
import ast
import yaml
import os
from google.oauth2 import service_account
import googleapiclient.discovery
from google.cloud import language_v1

In [258]:
# Create fetch url in this block
def create_twitter_url():
    handle = "MarianaSeafood"
    max_results = 10
    mrf = "max_results={}".format(max_results)
    q = "query=from:{}".format(handle)
    url = "https://api.twitter.com/2/tweets/search/recent?{}&{}".format(
        mrf, q
    )
    return url

In [259]:
# fetch key from yaml file in this block
def process_yaml():
    with open("config.yaml") as file:
        return yaml.safe_load(file)

In [260]:
def create_bearer_token_t(data):
    return data["search_tweets_api"]["bearer_token"]

In [261]:
def create_bearer_token_g(data):
    return [data["search_google_api"]["service_account_email"],data["search_google_api"]["bearer_token"]]

In [262]:
def twitter_auth_and_connect(bearer_token, url):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    # print(headers)
    response = requests.request("GET", url, headers=headers)
    return response.json()

In [263]:
#shape the data for analysis
def lang_data_shape(res_json):
    data_only = res_json["data"]
    doc_start = '"documents": {}'.format(data_only)
    str_json = "{" + doc_start + "}"
    # print(str_json)
    dump_doc = json.dumps(str_json)
    # print(dump_doc)
    doc = json.loads(dump_doc)
    # print(doc)
    return ast.literal_eval(doc)

In [264]:
'''
def create_key(service_account_email):
    """Creates a key for a service account."""

    credentials = service_account.Credentials.from_service_account_file(
        filename=os.environ['GOOGLE_APPLICATION_CREDENTIALS'],
        scopes=['https://www.googleapis.com/auth/cloud-platform'])

    service = googleapiclient.discovery.build(
        'iam', 'v1', credentials=credentials)

    key = service.projects().serviceAccounts().keys().create(
        name='projects/-/serviceAccounts/' + service_account_email, body={}
        ).execute()

    print('Created key: ' + key['name'])
    return(key['name'])
'''

In [269]:
def sample_analyze_sentiment(text_content):
    """
    Analyzing Sentiment in a String

    Args:
      text_content The text content to analyze
    """
    # client = language_v1.LanguageServiceClient()
    client = language_v1.LanguageServiceClient.from_service_account_json(
        "./pro-tuner-327300-09748beb1487.json")
    # text_content = 'I am so happy and joyful.'

    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type_": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = language_v1.EncodingType.UTF8

    response = client.analyze_sentiment(
        request={'document': document, 'encoding_type': encoding_type})
    # Get overall sentiment of the input document
    print(u"Document sentiment score: {}".format(
        response.document_sentiment.score))
    print(
        u"Document sentiment magnitude: {}".format(
            response.document_sentiment.magnitude
        )
    )
    # Get sentiment for all sentences in the document
    for sentence in response.sentences:
        print(u"Sentence text: {}".format(sentence.text.content))
        print(u"Sentence sentiment score: {}".format(sentence.sentiment.score))
        print(u"Sentence sentiment magnitude: {}".format(
            sentence.sentiment.magnitude))

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(u"Language of the text: {}".format(response.language))


In [270]:
def main():
    url = create_twitter_url()
    print("url=" + url)

    data = process_yaml()
    # print(type(data))
    # print("DATA=")
    # print(data)

    bearer_token_t = create_bearer_token_t(data)
    # print("Twitter TOken")
    # print(bearer_token_t)
    [account_g, bearer_token_g] = create_bearer_token_g(data)
    # print("Google TOken")
    # print(bearer_token_g)
    # print(account_g)

    res_json = twitter_auth_and_connect(bearer_token_t, url)
    print("json=")
    print(res_json)

    documents = lang_data_shape(res_json)
    # print("docu=")
    # print(documents)

    for items in documents['documents']:
        # print(items['text'])
        text_content = items['text']
        print("Text=")
        print(text_content)
        sample_analyze_sentiment(text_content)


if __name__ == "__main__":
    main()


url=https://api.twitter.com/2/tweets/search/recent?max_results=10&query=from:MarianaSeafood
json=
{'data': [{'id': '1447404383029841921', 'text': 'Testing twitter api'}, {'id': '1447396580504576017', 'text': 'The third test message'}], 'meta': {'newest_id': '1447404383029841921', 'oldest_id': '1447396580504576017', 'result_count': 2}}
Text=
Testing twitter api
Document sentiment score: 0.10000000149011612
Document sentiment magnitude: 0.10000000149011612
Sentence text: Testing twitter api
Sentence sentiment score: 0.10000000149011612
Sentence sentiment magnitude: 0.10000000149011612
Language of the text: en
Text=
The third test message
Document sentiment score: -0.10000000149011612
Document sentiment magnitude: 0.10000000149011612
Sentence text: The third test message
Sentence sentiment score: -0.10000000149011612
Sentence sentiment magnitude: 0.10000000149011612
Language of the text: en
